### Load Stuff

In [1]:
from kernel_submission import *
train_df = get_training_data('data/train.csv')
train_df = clean_data(train_df)
train_df = train_df.reset_index()[hh_columns+['idhogar', 'Target']]
target_household_map = target_by_household(train_df)

/home/minnie/.pyenv/versions/3.6.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/minnie/.pyenv/versions/3.6.6/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
import scipy
train_df = train_df.drop(target_column, axis=1).groupby(household_id).agg(lambda x: scipy.stats.mode(x)[0])

/home/minnie/.ve/poverty-prediction/lib/python3.6/site-packages/scipy/stats/stats.py:245: RuntimeWarning: The input array could not be properly checked for nan values. nan values will be ignored.
  "values. nan values will be ignored.", RuntimeWarning)


In [3]:
train_df = train_df.join(target_household_map)
train_df = compress_column_data(train_df)
train_df = add_custom_features(train_df)

/home/minnie/.ve/poverty-prediction/lib/python3.6/site-packages/pandas/core/computation/expressions.py:180: UserWarning: evaluating in Python space because the '+' operator is not supported by numexpr for the bool dtype, use '|' instead
  .format(op=op_str, alt_op=unsupported[op_str]))


In [4]:
is_extreme = convert_to_binary_targets(train_df, 1)

In [5]:
is_extreme['Target'].value_counts()

0    2766
1     222
Name: Target, dtype: int64

In [6]:
# from sklearn.metrics import f1_score
# f1_score(y_true, y_predicted, average = 'macro`)

### Plot Stuff

In [7]:
# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Set a few plotting defaults
%matplotlib inline
plt.style.use('fivethirtyeight')
plt.rcParams['font.size'] = 12
plt.rcParams['patch.edgecolor'] = 'k'

In [8]:
train = is_extreme
train['v2a1'] = train['v2a1'].astype(float)

In [9]:
# from collections import OrderedDict
# from numpy.linalg import LinAlgError

# plt.figure(figsize = (20, 4))
# plt.style.use('fivethirtyeight')

# # Color mapping
# colors = OrderedDict({0: 'blue', 1: 'green'})
# poverty_mapping = OrderedDict({1: 'extreme', 0: 'not extreme'})

# # Iterate through the float columns
# for i, col in enumerate(train.select_dtypes('float')):
#     try:
#         ax = plt.subplot(2, 2, i + 1)
#         # Iterate through the poverty levels
#         for poverty_level, color in colors.items():
#             # Plot each poverty level as a separate line
#             sns.kdeplot(train.loc[train['Target'] == poverty_level, col].dropna(), 
#                         ax = ax, color = color, label = poverty_mapping[poverty_level])

#         plt.title(f'{col.capitalize()} Distribution'); plt.xlabel(f'{col}'); plt.ylabel('Density')
#     except LinAlgError:
#         pass

# plt.subplots_adjust(top = 2)

In [10]:
# train.select_dtypes(np.int64).nunique().value_counts().sort_index().plot.bar(color = 'blue', 
#                                                                              figsize = (6, 4),
#                                                                             edgecolor = 'k', linewidth = 1)
# plt.xlabel('Number of Unique Values'); plt.ylabel('Count')
# plt.title('Count of Unique Values in Integer Columns')

### Find way of scaling household material against wealth

#### Exploration

In [11]:
def plot_categoricals(x, y, data, annotate = True):
    """Plot counts of two categoricals.
    Size is raw count for each grouping.
    Percentages are for a given value of y."""
    
    # Raw counts 
    raw_counts = pd.DataFrame(data.groupby(y)[x].value_counts(normalize = False))
    raw_counts = raw_counts.rename(columns = {x: 'raw_count'})
    
    # Calculate counts for each group of x and y
    counts = pd.DataFrame(data.groupby(y)[x].value_counts(normalize = True))
    
    # Rename the column and reset the index
    counts = counts.rename(columns = {x: 'normalized_count'}).reset_index()
    counts['percent'] = 100 * counts['normalized_count']
    
    # Add the raw count
    counts['raw_count'] = list(raw_counts['raw_count'])
    
    plt.figure(figsize = (14, 10))
    # Scatter plot sized by percent
    plt.scatter(counts[x], counts[y], edgecolor = 'k', color = 'lightgreen',
                s = 100 * np.sqrt(counts['raw_count']), marker = 'o',
                alpha = 0.6, linewidth = 1.5)
    
    if annotate:
        # Annotate the plot with text
        for i, row in counts.iterrows():
            # Put text with appropriate offsets
            plt.annotate(xy = (row[x] - (1 / counts[x].nunique()), 
                               row[y] - (0.15 / counts[y].nunique())),
                         color = 'navy',
                         s = f"{round(row['percent'], 1)}%")
        
    # Set tick marks
    plt.yticks(counts[y].unique())
    plt.xticks(counts[x].unique())
    
    # Transform min and max to evenly space in square root domain
    sqr_min = int(np.sqrt(raw_counts['raw_count'].min()))
    sqr_max = int(np.sqrt(raw_counts['raw_count'].max()))
    
    # 5 sizes for legend
    msizes = list(range(sqr_min, sqr_max,
                        int(( sqr_max - sqr_min) / 5)))
    markers = []
    
    # Markers for legend
    for size in msizes:
        markers.append(plt.scatter([], [], s = 100 * size, 
                                   label = f'{int(round(np.square(size) / 100) * 100)}', 
                                   color = 'lightgreen',
                                   alpha = 0.6, edgecolor = 'k', linewidth = 1.5))
        
    # Legend and formatting
    plt.legend(handles = markers, title = 'Counts',
               labelspacing = 3, handletextpad = 2,
               fontsize = 16,
               loc = (1.10, 0.19))
    
    plt.annotate(f'* Size represents raw count while % is for a given y value.',
                 xy = (0, 1), xycoords = 'figure points', size = 10)
    
    # Adjust axes limits
    plt.xlim((counts[x].min() - (6 / counts[x].nunique()), 
              counts[x].max() + (6 / counts[x].nunique())))
    plt.ylim((counts[y].min() - (4 / counts[y].nunique()), 
              counts[y].max() + (4 / counts[y].nunique())))
    plt.grid(None)
    plt.xlabel(f"{x}"); plt.ylabel(f"{y}"); plt.title(f"{y} vs {x}");

In [12]:
# material = ['elimbasu3',  'elimbasu2', 'elimbasu5', 'elimbasu1', 'elimbasu4', 'elimbasu6']
material = ['elimbasu3', 'elimbasu2', 'elimbasu5', 'elimbasu1', 'elimbasu4', 'elimbasu6']
features = train[material]

KeyError: "['elimbasu3' 'elimbasu2' 'elimbasu5' 'elimbasu1' 'elimbasu4' 'elimbasu6'] not in index"

In [ ]:
features = compress_columns(features, 'material', material).join(train['Target'])

In [ ]:
features = get_balanced_data(features, 222, 6)
plot_categoricals('Target','material', features)

In [ ]:
desc = {'elimbasu1':'=1 if rubbish disposal mainly by tanker truck',
    'elimbasu2':'=1 if rubbish disposal mainly by botan hollow or buried',
    'elimbasu3':'=1 if rubbish disposal mainly by burning',
    'elimbasu4':'=1 if rubbish disposal mainly by throwing in an unoccupied space',
    'elimbasu5':'"=1 if rubbish disposal mainly by throwing in river,  creek or sea"',
    'elimbasu6':'=1 if rubbish disposal mainly other'}
desc.keys()

In [ ]:
for m in material:
    print(desc.get(m))

#### Replacements

In [ ]:
# df = compress_columns(df, 'wall-material', ['paredfibras', 'pareddes', 'paredzinc', 'paredzocalo', 'paredmad', 'paredpreb', 'paredblolad', 'paredother'])
# df = compress_columns(df, 'floor-material', ['pisonotiene', 'pisonatur', 'pisomadera', 'pisocemento', 'pisomoscer', 'pisoother'])
# df = compress_columns(df, 'roof-material', ['techocane', 'techoentrepiso', 'techozinc', 'techootro'])
# df = compress_columns(df, 'water-provision', ['abastaguano', 'abastaguafuera','abastaguadentro'])
# df = compress_columns(df, 'house-ownership', ['tipovivi4', 'tipovivi5', 'tipovivi1', 'tipovivi3', 'tipovivi2'])
# df = compress_columns(df, 'toilet-system', ['sanitario5', 'sanitario6', 'sanitario1', 'sanitario3', 'sanitario2'])
# df = compress_columns(df, 'cooking-energy', ['energcocinar4', 'energcocinar1', 'energcocinar3', 'energcocinar2'])
# df = compress_columns(df, 'rubbish-disposal', ['elimbasu3', 'elimbasu2', 'elimbasu5', 'elimbasu1', 'elimbasu4', 'elimbasu6'])

### Train Classifier

In [13]:
is_extreme['v2a1'] = is_extreme['v2a1'].astype(float)
is_extreme['warning'] = is_extreme['warning'].astype(int)

In [14]:
valid_df = get_balanced_data(is_extreme, 50)
train_df = pd.concat([is_extreme.reset_index(), valid_df.reset_index()]).drop_duplicates(keep=False).set_index(household_id)

In [15]:
valid_df = valid_df.drop(columns=['cielorazo','r4m1','hogar_nin'])
train_df = train_df.drop(columns=['cielorazo','r4m1','hogar_nin'])

In [16]:
train_df['Target'].value_counts()

0    2716
1     172
Name: Target, dtype: int64

In [17]:
train_df = get_balanced_data(train_df, 172)

In [18]:
print('Most negatively correlated variables:')
print(train_df.corr()['Target'].sort_values().head())

print('\nMost positively correlated variables:')
print(train_df.corr()['Target'].sort_values().dropna().tail())

Most negatively correlated variables:
edjefe           -0.319729
wrf              -0.308317
meaneduc         -0.301676
wall-quality     -0.297905
cooking-energy   -0.276121
Name: Target, dtype: float64

Most positively correlated variables:
r4h1            0.222420
warning         0.287465
r4t1            0.298408
overcrowding    0.321672
Target          1.000000
Name: Target, dtype: float64


### Train!

In [19]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, make_scorer
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import Imputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.pipeline import Pipeline

# Custom scorer for cross validation
scorer = make_scorer(f1_score, greater_is_better=True, average = 'macro')

/home/minnie/.ve/poverty-prediction/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [20]:
# Labels for training
train_labels = np.array(list(train_df['Target'].astype(np.uint8)))

# Extract the training data
train_set = train_df.drop(columns = ['Id', 'Target'])
test_set = valid_df.drop(columns = ['Id', 'Target'])

In [21]:
features = list(train_set.columns)

pipeline = Pipeline([('imputer', Imputer(strategy = 'median')), 
                      ('scaler', MinMaxScaler())])

# Fit and transform training data
train_set = pipeline.fit_transform(train_set)
test_set = pipeline.transform(test_set)

In [22]:
model = RandomForestClassifier(n_estimators=100, random_state=10, n_jobs = -1)
# 10 fold cross validation
cv_score = cross_val_score(model, train_set, train_labels, cv = 10, scoring = scorer)

print(f'10 Fold Cross Validation F1 Score = {round(cv_score.mean(), 4)} with std = {round(cv_score.std(), 4)}')

10 Fold Cross Validation F1 Score = 0.7464 with std = 0.0966


In [23]:
from sklearn.metrics import classification_report
clf = RandomForestClassifier(n_estimators=100, random_state=10, n_jobs = -1)
clf.fit(train_set, train_labels)
preds = clf.predict(test_set)
print(classification_report(valid_df['Target'], preds))

             precision    recall  f1-score   support

          0       0.74      0.64      0.69        50
          1       0.68      0.78      0.73        50

avg / total       0.71      0.71      0.71       100



In [24]:
class_probabilities = clf.predict_proba(test_set)

In [25]:
class_probabilities

array([[0.36, 0.64],
       [0.28, 0.72],
       [0.81, 0.19],
       [0.28, 0.72],
       [0.45, 0.55],
       [0.86, 0.14],
       [0.42, 0.58],
       [0.24, 0.76],
       [0.77, 0.23],
       [0.86, 0.14],
       [0.67, 0.33],
       [0.83, 0.17],
       [0.34, 0.66],
       [0.92, 0.08],
       [0.88, 0.12],
       [0.5 , 0.5 ],
       [0.48, 0.52],
       [0.95, 0.05],
       [0.37, 0.63],
       [0.9 , 0.1 ],
       [0.55, 0.45],
       [0.55, 0.45],
       [0.55, 0.45],
       [0.89, 0.11],
       [0.65, 0.35],
       [0.44, 0.56],
       [0.32, 0.68],
       [0.27, 0.73],
       [0.8 , 0.2 ],
       [0.88, 0.12],
       [0.32, 0.68],
       [0.97, 0.03],
       [0.66, 0.34],
       [0.66, 0.34],
       [0.2 , 0.8 ],
       [0.29, 0.71],
       [0.8 , 0.2 ],
       [0.05, 0.95],
       [0.71, 0.29],
       [0.43, 0.57],
       [0.24, 0.76],
       [0.85, 0.15],
       [0.53, 0.47],
       [0.68, 0.32],
       [0.45, 0.55],
       [0.26, 0.74],
       [0.8 , 0.2 ],
       [0.94,